In [210]:
import serial
import re
import time
import pygame
import sys
import numpy as np
import math

In [211]:
class Position:
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0

class Orientation:
    def __init__(self):
        self.roll = 0
        self.pitch = 0

class Pose:
    def __init__(self):
        self.position = Position()
        self.orientation = Orientation()

    def update_pose(self, pose_array):
        self.position.x = pose_array[0]
        self.position.y = pose_array[1]
        self.position.z = pose_array[2]
        self.orientation.pitch = pose_array[3]
        self.orientation.roll = pose_array[4]

    def __str__(self):
        return f"X: {self.position.x}, Y: {self.position.y}, Z: {self.position.z}, P: {self.orientation.pitch}, R: {self.orientation.roll}"

In [212]:
class Robot:
    def __init__(self, port):
        self.ser = serial.Serial(port)
        self.ser.write(b"CON \r")
        time.sleep(1)


        self.set_mode('manual')
        self.set_movement_mode('J')
                
        time.sleep(1)
        self.ser.write(b"S\r")
        time.sleep(0.2)
        self.ser.write(b"5\r")
        
        self.commands = {
            'x': {
                'increase': '1',
                'decrease': 'Q',
            },
            'y': {
                'increase': '2',
                'decrease': 'W',
            },
            'z': {
                'increase': '3',
                'decrease': 'E',
            },
            'p': {
                'increase': '4',
                'decrease': 'R',
            },
            'r': {
                'increase': '5',
                'decrease': 'T',
            },
        }

    def close(self):
        self.ser.close()

    def home(self, port):
        self.ser.write(b"HOME \r")
        time.sleep(180)

    def set_movement_mode(self, mode):
        self.movement_mode = mode
        if mode == 'J':
            self.ser.write(b"J\r")
        if mode == 'X':
            self.ser.write(b"X\r")

    def set_mode(self, mode):
        self.mode = mode
        self.ser.write(b"~\r")
        
        if mode == "manual":
            desired_command = "MANUAL MODE!"
            undesired_command = "EXIT"
        elif mode == "auto":
            desired_command = "EXIT"
            undesired_command = "MANUAL MODE!"

        check = ''
        start_time = time.time()
        while True:
            check += self.ser.read_all().decode('ascii')
            print(check)

            if bool(re.search(undesired_command, check)):
                self.ser.write(b"~\r")
                return
            if bool(re.search(desired_command, check)):
                return

            if time.time() - start_time > 10: 
                raise TimeoutError

            time.sleep(0.05)

        if bool(re.search("DISABLED", check)) or bool(re.search("IMPACT", response)):
            if self.mode == 'manual':
                self.ser.write(b"C \r")
            else:
                self.ser.write(b"CON \r")


    def move_manual(self, axis: str, direction: str):
        if direction not in ['increase', 'decrease']:
            raise ValueError("Direction must be 'increase' or 'decrease'")

        if axis not in ['x', 'y', 'z', 'p', 'r']:
            raise ValueError("Axis must be 'x', 'y', 'z', 'p' (pitch) or 'r' (roll)")

        if axis in ['p', 'r'] and self.movement_mode == 'X':
            self.set_movement_mode('J')
        if axis in ['x', 'y', 'z'] and self.movement_mode == 'J':
            self.set_movement_mode('X')

        command_to_send = self.commands[axis][direction]
        self.ser.write(bytes(f"{command_to_send} \r", encoding='utf-8'))
        self.check_messages()
    
    def check_messages(self):
        response = self.ser.read_all().decode('ascii')
        if bool(re.search("DISABLED", response)) or bool(re.search("IMPACT", response)):
            if self.mode == 'manual':
                self.ser.write(b"C \r")
            else:
                self.ser.write(b"CON \r")
                
        print(response)

    def print_pose(self):
        print(self.pose)

In [213]:
class Controller:
    def __init__(self, idx):
        self.joystick = pygame.joystick.Joystick(0)
        self.joystick.init()
        self.threshold = 0.9
        self.axes = []
        self.buttons = []

    def get_input_state(self):
        self.axes = [round(self.joystick.get_axis(i), 1) for i in range(self.joystick.get_numaxes())]
        self.buttons = [self.joystick.get_button(i) == 1 for i in range(self.joystick.get_numbuttons())]

        return self.axes, self.buttons

In [214]:
def process_input(axes, buttons, robot_bisturi):
    if buttons[0]:
        print("camera position 1")
    elif buttons[1]:
        print("camera position 2")
    elif buttons[2]:
        print("camera position 3")
    elif buttons[3]:
        print("camera position 4")

    if buttons[4] and not buttons[6]:
        robot_bisturi.move_manual('z', 'decrease')
        print("Decrease z")
    elif not buttons[4] and buttons[6]:
        robot_bisturi.move_manual('z', 'increase')
        print("Increase z")

    if axes[0] >= threshold:
        robot_bisturi.move_manual('y', 'increase')
        print("Increase y")
    elif axes[0] <= -threshold:
        robot_bisturi.move_manual('y', 'decrease')
        print("Decrease y")

    if axes[1] >= threshold:
        robot_bisturi.move_manual('x', 'decrease')
        print("Decrease x")
    elif axes[1] <= -threshold:
        robot_bisturi.move_manual('x', 'increase')
        print("Increase x")

    if axes[2] >= threshold:
        robot_bisturi.move_manual('r', 'increase')
        print("Increase roll")
    elif axes[2] <= -threshold:
        robot_bisturi.move_manual('r', 'decrease')
        print("Decrease roll")

    if axes[3] >= threshold:
        robot_bisturi.move_manual('p', 'decrease')
        print("Decrease pitch")
    elif axes[3] <= -threshold:
        robot_bisturi.move_manual('p', 'increase')
        print("Increase pitch")



In [215]:
pygame.init()
pygame.joystick.init()

robot_bisturi = Robot("COM12")

controller = Controller(0)

CONTROL ENABLED.                   
*** IMPACT PROTECTION axis 9
>
CONTROL ENABLED.                   
*** IMPACT PROTECTION axis 9
>
EXIT MANUAL MODE...
>
>


In [216]:

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            robot.close()
            pygame.quit()
    
    axes, buttons = controller.get_input_state()
    
    process_input(axes, buttons, robot_bisturi)

    time.sleep(0.05)


MANUAL MODE!
>
SPEED..5                           
 XYZ MODE.                
Increase x
 XYZ MODE.               
Increase x
 
Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Increase x

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y

Decrease y
